In [4]:
%cd D:\proj\recept\photos
%matplotlib inline
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import random
from skimage.morphology import binary_dilation, binary_erosion
import os, numpy, PIL
from PIL import Image
import scipy.ndimage
import imutils
import glob

D:\proj\recept\photos


In [5]:
name_images = "photo_1000menu_"
ext = ".jpg"

def getCountFile():
    return len(glob.glob1('',name_images+"*"+ext))

count_photo = getCountFile()

In [6]:
def show(img):
    plt.figure(figsize = (20,10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), interpolation='nearest')
    plt.show()

In [7]:
def minimal_size_img(treshold):
    minimal_size = (9999,9999)
    for i in range(count_photo):
        size = Image.open(name_images+str(i+1)+ext).size
        if size < minimal_size and size > treshold:
            minimal_size = size
    return minimal_size

In [9]:
size_img = minimal_size_img((557, 390))

In [10]:
PHOTO_X = size_img[0]
PHOTO_Y = size_img[1]

In [12]:
def resise_photo(name_work_dir):
    if not os.path.exists(name_work_dir):
        os.mkdir(name_work_dir, mode=0o777) 
    for i in range(count_photo):
        image = Image.open(name_images+str(i+1)+ext)
        new_image = image.resize((PHOTO_X, PHOTO_Y))

        new_image.save(name_work_dir+'/'+name_images+str(i+1)+ext)

In [14]:
work_dir = name_images+"_work"
resise_photo(work_dir)

In [15]:
def calulate_avg_photo():
    allfiles=os.listdir(os.getcwd()+'/'+work_dir)
    imlist=[work_dir+'/'+filename for filename in allfiles if  filename[-4:] in [ext]]
    w,h=Image.open(imlist[5]).size
    
    N=len(imlist)
    
    arr=numpy.zeros((h,w,3),numpy.float)
    
    for im in imlist:
        imarr=numpy.array(Image.open(im),dtype=numpy.float)
        arr=arr+imarr/N
    
    arr=numpy.array(numpy.round(arr),dtype=numpy.uint8)
    
    out=Image.fromarray(arr)
    out.save(work_dir+'/'+"Average.png")

In [16]:
calulate_avg_photo()

580 396
(396, 580, 3)


In [19]:
def calculate_mask():
    fn = work_dir+'/Average.png'
    kernel = np.ones((5,5),np.uint8)
    img = cv.imread(fn)
    image = img.copy()
    image = imutils.resize(image, height = PHOTO_Y, width=PHOTO_X)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    gray = cv.bilateralFilter(gray, 1, 1, 17)
    # show(gray)
    grayBlur = cv.GaussianBlur(gray,(1,1),0 )
    # show(grayBlur)
    edged = cv.Canny(grayBlur, 0, 25,1)
    # show(edged)
    edged = cv.dilate(edged,kernel,iterations = 1)
    # show(edged)

    out=Image.fromarray(edged)
    out.save(work_dir+'/'+"mask_"+name_images+".png")

In [20]:
calculate_mask()

In [34]:
def deleted_watermark(path_photo):
    img = cv.imread(work_dir+'/'+path_photo+'.jpg')
#     show(img)
    # image = Image.open('photo_1000menu_2.jpg')
    # new_image = image.resize((600, 600))
    new_image = numpy.array(img) 
    mask = cv.imread(work_dir+'/'+"mask_"+name_images+".png",0)
    # image.shape, mask.shape
    dst = cv.inpaint(img,mask,3,cv.INPAINT_TELEA)
    open_cv_image = dst[:, :, ::-1].copy()
    if not os.path.exists("handle_images"):
        os.mkdir("handle_images", mode=0o777) 
    im_pil = Image.fromarray(open_cv_image)
    im_pil.save('handle_images/'+path_photo+ext) 
#     show(dst)

In [35]:
deleted_watermark('photo_1000menu_9')

In [36]:
# def delete_work_dir():
#     os.rmdir(work_dir)

In [1]:
# delete_work_dir()